# 03 - Deploy and Run the Training Pipeline to Vertex AI

The notebook covers the following tasks:

1. Build the container image for the ETL and Training steps.
2. Compile and upload the KFP pipelne to `Vertex AI`.
2. Run the KFP pipeline using `Vertex Pipelines`.
3. Execute the pipeline deployment CI/CD steps using `Cloud Build`.

## Setup

In [ ]:
import os
import kfp
from src.kfp_pipelines import runner

print("KFP Version:", kfp.__version__)

In [ ]:
PROJECT = 'merlin-on-gcp'
REGION = 'us-central1'
BUCKET = 'merlin-on-gcp'
VERTEX_SERVICE_ACCOUNT = f'vertex-sa-mlops@{PROJECT}.iam.gserviceaccount.com'

VERSION = 'v1'
MOVIES_DATASET_DISPLAY_NAME = 'movielens25m-movies'
RATINGS_DATASET_DISPLAY_NAME = 'movielens25m-ratings'
MODEL_DISPLAY_NAME = f'movielens25m-recommender-{VERSION}'
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}-train-pipeline'

## 1. Build Container Image

In [ ]:
IMAGE_NAME = "movielens-cuda11.0-tf2.4"
IMAGE_URI = f"gcr.io/{PROJECT}/{IMAGE_NAME}"
print(IMAGE_URI)

In [ ]:
! gcloud builds submit --tag {IMAGE_URI} . --timeout=45m --machine-type=e2-highcpu-8

## 2. Compile and upload the KFP Pipeline 

### Set pipeline configurations

In [ ]:
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] =  REGION
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/movielens25m"
os.environ["IMAGE_URI"] = IMAGE_URI

In [ ]:
from src.kfp_pipelines import config
import importlib
importlib.reload(config)

for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

### Compile pipeline

In [ ]:
pipeline_definition_file = f'{config.PIPELINE_NAME}.json'
runner.compile_pipeline(pipeline_definition_file)

### Upload pipeline to GCS

In [ ]:
PIPELINES_STORE = os.path.join(config.GCS_LOCATION, "compilies_pipelines/")

! gsutil cp {pipeline_definition_file} {PIPELINES_STORE}
! gsutil ls {PIPELINES_STORE}

## 3. Run the KFP pipeline using Vertex Pipelines

In [ ]:
parameter_values = {}
gcs_pipeline_definition_file = os.path.join(PIPELINES_STORE, pipeline_definition_file)

_ = runner.run_pipeline(
    gcs_pipeline_definition_file,
    parameter_values=parameter_values,
)

## 4. Execute the pipeline deployment CI/CD steps using Cloud Build.